In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from copy import copy

cap = cv2.VideoCapture("vid1_IR.avi")
i = 0
step = 1
thresh = 38

while cap.isOpened():
    ret, frame = cap.read()
    if ret and not i % 5:
        G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow("IR", G)
        _, Ibin = cv2.threshold(G, thresh, 255, cv2.THRESH_BINARY)
        cv2.imshow("bin", Ibin)
        Ibin = cv2.medianBlur(Ibin, 7)
        Imorph = cv2.morphologyEx(Ibin, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
        Imorph = cv2.morphologyEx(Imorph, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7)))
        cv2.imshow("morph", Imorph)

        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(Imorph)
        # retval -- total number of unique labels
        # labels -- destination labeled image
        # stats -- statistics output for each label , including the background label .
        # centroids -- centroid output for each label , including the background label .
        cv2.imshow("Labels", np.uint8(labels / retval * 255))
        

        min_area = 100  # Minimalna powierzchnia sylwetki lub jej elementu
        aspect_ratio_thresh = 0.9  # Minimalny stosunek wysokości do szerokości dla sylwetek i ich elementów
        max_vertical_gap = 50  # Maksymalna pionowa odległość między sylwetkami do połączenia

        Irect = cv2.cvtColor(Imorph, cv2.COLOR_GRAY2BGR)

        filtered_components = []  # prostokąty dla obiektów po filtracji w formacie [(x, y, w, h), ...]
        # (a) Rysuj wszystkie prostokąty – zielone
        for x, y, w, h, area in stats[1:]:  # Pomijamy tło (label 0)

            cv2.rectangle(Irect, (x, y), (x + w, y + h), (0, 0, 255), 1)  # zaznaczanie wszystkich wykrytych obiektów

            if area < min_area:
                continue

            # (c) Filtracja po proporcjach (wysokość / szerokość)
            aspect_ratio = h / float(w)
            if aspect_ratio < aspect_ratio_thresh:
                continue

            # Jeśli przeszło – dodajemy do filtrowanych
            filtered_components.append((x, y, w, h))
            
        for x, y, w, h in filtered_components:
            cv2.rectangle(Irect, (x, y), (x + w, y + h), (0, 255, 0), 1)

        # (d) Łączenie obiektów leżących jeden pod drugim
        merged_components = []
        components_queue = copy(filtered_components)
        
        while components_queue:
            x1, y1, w1, h1 = components_queue.pop(0)
            rect1 = (x1, y1, x1 + w1, y1 + h1)
            merged = False

            for j, (x2, y2, w2, h2) in enumerate(components_queue):
                rect2 = (x2, y2, x2 + w2, y2 + h2)
                horizontal_overlap = not (rect2[2] < rect1[0] or rect2[0] > rect1[2])
                bottom1 = y1 + h1
                bottom2 = y2 + h2
                if bottom1 <= y2:
                    vertical_alignment = (y2 - bottom1) <= max_vertical_gap
                elif bottom2 <= y1:
                    vertical_alignment = (y1 - bottom2) <= max_vertical_gap
                else:
                    vertical_alignment = True # Zachodzą na siebie w pionie 

                if horizontal_overlap and vertical_alignment:
                    # Łączymy prostokąty
                    x_min = min(x1, x2)
                    y_min = min(y1, y2)
                    x_max = max(x1 + w1, x2 + w2)
                    y_max = max(y1 + h1, y2 + h2)
                    merged_obj = (x_min, y_min, x_max - x_min, y_max - y_min)
                    merged = True
                    break

            if not merged:
                merged_components.append((x1, y1, w1, h1))
            else:
                # Po scaleniu pomijamy już porównane obiekty
                components_queue.pop(j)
                # Chcemy dalej szukać połączeń między nowymi obiektami
                components_queue.append(merged_obj)

        # Rysuj połączone sylwetki – niebieskie grube prostokąty
        for x, y, w, h in merged_components:
            cv2.rectangle(Irect, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Pokaż wynik
        cv2.imshow("Merging", Irect)
        
    if (cv2.waitKey(10) & 0xFF == ord("q")) or not ret:  # break the loop when the ’q’ key is pressed
        break
    i += 1

cap.release()
cv2.destroyAllWindows()